# Indexing Data
---
Prepare policy data and employee single view data for AI agent.

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

## Policy Data
---
I will prepare policy data as vector database and then convert it to retrieval tool for AI agent.

In [3]:
with open('../data/input/attendance_policy.md', 'r') as f:
    policy_text = f.read()
print(len(policy_text))

10272


In [4]:
print(policy_text[:1000])

**Company Attendance Policy**

**1. Purpose and Scope**  
The Attendance Policy aims to set clear expectations for employee attendance, promote a productive work environment, and ensure fairness and consistency across the organization. Attendance is a critical factor that impacts the workflow, team dynamics, and overall company performance. This policy applies to all full-time, part-time, and contract employees, regardless of department or seniority, and ensures that all individuals are aware of their responsibilities regarding attendance.

**2. General Expectations**  
Employees are expected to maintain regular, punctual attendance to ensure smooth workflow and minimal disruption to team activities. Consistent attendance is essential for both individual performance and overall team productivity. Employees should report to work on time, be ready to fulfill their responsibilities during assigned hours, and demonstrate reliability by maintaining a predictable work schedule. If an employe

In [5]:
import re

In [6]:
pattern = r'(\*\*\d+\. [^*]+\*\*[^*]+(?:\n[^*]+)*)'
matches = re.findall(pattern, policy_text)

In [7]:
# expected 13 policies
len(matches)

13

In [8]:
for m in matches[:3]:
    print(m.strip())
    print("-" * 20)

**1. Purpose and Scope**  
The Attendance Policy aims to set clear expectations for employee attendance, promote a productive work environment, and ensure fairness and consistency across the organization. Attendance is a critical factor that impacts the workflow, team dynamics, and overall company performance. This policy applies to all full-time, part-time, and contract employees, regardless of department or seniority, and ensures that all individuals are aware of their responsibilities regarding attendance.
--------------------
**2. General Expectations**  
Employees are expected to maintain regular, punctual attendance to ensure smooth workflow and minimal disruption to team activities. Consistent attendance is essential for both individual performance and overall team productivity. Employees should report to work on time, be ready to fulfill their responsibilities during assigned hours, and demonstrate reliability by maintaining a predictable work schedule. If an employee foresees 

In [9]:
policy_chunks = [m.strip() for m in matches]

In [10]:
from langchain_core.documents import Document

In [11]:
policy_docs = []
for c in policy_chunks:
    doc = Document(page_content=c)
    policy_docs.append(doc)

print(len(policy_docs))

13


In [12]:
policy_docs[0]

Document(metadata={}, page_content='**1. Purpose and Scope**  \nThe Attendance Policy aims to set clear expectations for employee attendance, promote a productive work environment, and ensure fairness and consistency across the organization. Attendance is a critical factor that impacts the workflow, team dynamics, and overall company performance. This policy applies to all full-time, part-time, and contract employees, regardless of department or seniority, and ensures that all individuals are aware of their responsibilities regarding attendance.')

In [13]:
from langchain_openai import AzureOpenAIEmbeddings

In [14]:
# initialize
openai_api_version = "2024-02-01"
embedding_azure_deployment = "text-embedding-3-large"
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embedding_azure_deployment,
    openai_api_version=openai_api_version,
)

In [15]:
from langchain_chroma import Chroma

In [16]:
vector_store = Chroma(
    collection_name="attendance_policy_v3",
    embedding_function=embeddings,
    persist_directory="./vector_store",
)

In [17]:
vector_store.add_documents(policy_docs)

['9bfe02d3-a785-430a-a5c4-f8a3d501b277',
 '5544c0af-77ce-4221-8063-3540d8c0e256',
 'ad7bbfba-7699-4f52-8eb8-3d1987538fce',
 '6d9ba569-d3a9-49f7-9bfd-b28a00a2b54f',
 '1f70f67b-46f9-473b-a4b5-45486a1579f0',
 'daf6bf3a-6b4a-483e-b9a6-fcc10a581bd9',
 '577b0f28-3136-4243-a98e-8604a33debe0',
 'd96e337d-e392-4c3a-ae81-c3028eb8ae15',
 '22f14aae-0ccb-49aa-9a59-b3f2bc478608',
 'ab653aa0-13eb-4850-8fe1-a37f39420393',
 '4f18852e-3a59-45c7-bf59-39f91fa7ac90',
 '3b481330-8492-47be-bfa4-043522518942',
 'f7503c30-9a3f-4d06-831f-1e7c1ed8f2f1']

In [18]:
vector_store._collection.count()

13

## Employee Single View
---
LangChain recommends to interact with CSV data by SQL then I will crete `sqlite` database for the employee single view data.

In [3]:
import pandas as pd

In [5]:
employee_df = pd.read_excel("../data/input/single_view_with_attrition_score.xlsx")
employee_df

,emp_id,emp_firstname,emp_lastname,team_id,team_name,department_id,department_name,card_id,on_site_days,is_on_site,...,performance_last_3_months,performance_last_6_months,performance_last_12_months,performance_change_3_to_12_months,performance_grade,last_performance_grade,resign,attrition_score,risk_group,resign_in_3_months
0,1,Robert,Price,7,QA,8,Customer Support,4835012087320540,135,1,...,98.98,78.80,71.51,14.62,D,A,True,0.799646,Very High Risk,True
1,2,Frances,Cochran,1,AI,2,Engineering,4657016181544010,77,1,...,86.35,54.52,54.48,-14.77,B,F,False,0.029061,Low Risk,False
2,3,Jonathan,Perry,9,Security,6,Product,4736011345354630,148,1,...,53.66,91.83,94.40,-1.80,F,D,False,0.607111,High Risk,False
3,4,Jon,Crawford,8,Mobile,5,Finance,4718662102507140,188,1,...,77.96,77.02,59.11,14.33,B,C,False,0.027803,Low Risk,False
4,5,Daniel,Smith,1,AI,5,Finance,4354103335855230,104,1,...,59.52,83.14,76.53,13.46,F,A,True,0.920683,Very High Risk,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,Cynthia,Barnes,4,Backend,4,Marketing,4391645925253450,181,1,...,62.51,91.46,59.02,-17.05,C,D,True,0.891173,Very High Risk,True
196,197,Wayne,Curtis,5,Data Science,5,Finance,4164274234036860,103,0,...,67.18,94.06,84.60,4.94,B,F,False,0.054471,Low Risk,False
197,198,Jorge,Miller,7,QA,8,Customer Support,4910751946941610,184,0,...,87.21,88.91,97.01,-2.80,B,B,False,0.044627,Low Risk,False
198,199,Cory,Burke,5,Data Science,6,Product,4503801837460110,105,1,...,76.62,83.05,90.92,-7.86,D,D,False,0.302252,Moderate Risk,False


In [6]:
employee_df.columns

Index(['emp_id', 'emp_firstname', 'emp_lastname', 'team_id', 'team_name',
       'department_id', 'department_name', 'card_id', 'on_site_days',
       'is_on_site', 'sprint_id', 'is_on_target', 'last_performance_score',
       'performance_score', 'sprint_start_date', 'sprint_end_date',
       'sprint_goal', 'sprint_success_rate', 'is_sprint_on_target',
       'total_check_ins', 'total_check_outs', 'attendance_consistency',
       'performance_rolling_avg', 'low_on_site_flag', 'sprints_participated',
       'avg_sprint_success_rate', 'performance_last_3_months',
       'performance_last_6_months', 'performance_last_12_months',
       'performance_change_3_to_12_months', 'performance_grade',
       'last_performance_grade', 'resign', 'attrition_score', 'risk_group',
       'resign_in_3_months'],
      dtype='object')

In [7]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

In [8]:
# SQLite connects to file-based databases
engine = create_engine("sqlite:///employee.db")

In [9]:
employee_df.to_sql('employee_single_view', con=engine, index=False, if_exists="replace")

200

In [10]:
db = SQLDatabase(engine=engine)

In [11]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM employee_single_view WHERE attrition_score > 0.8 LIMIT 1;"))

sqlite
['employee_single_view']
[(5, 'Daniel', 'Smith', 1, 'AI', 5, 'Finance', 4354103335855230, 104, 1, 5, 0, 91.65, 52.93, '2023-04-28 00:00:00.000000', None, 'Fire approach from fish then.', 0.98, 0, 75, 26, 0.62, 75.4, 1, 10, 0.77, 59.52, 83.14, 76.53, 13.46, 'F', 'A', 1, 0.920683026313782, 'Very High Risk', 1)]


In [12]:
print(db.run("SELECT count(*) FROM employee_single_view;"))

[(200,)]
